### Import libraries

In [1]:
from pyspark.sql import functions as F
import sys, os
path = os.path.dirname(os.getcwd()) + '/BaseUtils/'
sys.path.append(os.path.abspath(path))

from hdfs_io import *

hdfs_obj = HDFS_IO()

In [2]:
# Instanciate FLightRadarAPI

from FlightRadar24 import FlightRadar24API
fr_api = FlightRadar24API()

ModuleNotFoundError: No module named 'FlightRadar24'

### Get Zones

In [ ]:
zones = fr_api.get_zones()

In [ ]:
print(type(zones))
print(zones)

In [ ]:
zones_tuples = [{'name': zone} for zone in zones]
print(zones_tuples)

### Start Spark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import types as T

spark = SparkSession.builder \
    .appName("LocalSpark") \
    .master("spark://spark-master:7077") \
    .config("spark.rpc.message.maxSize", "1024") \
    .getOrCreate()

print("Spark version:", spark.version)

#### Create zones dataframe

In [ ]:
# Define the schema explicitly
zones_schema = T.StructType([
    T.StructField("name", T.StringType(), True)
])

In [ ]:
zones_df = spark.createDataFrame(zones_tuples, zones_schema)

#### Store to HDFS

In [ ]:
destination_path = hdfs_obj.base_url + hdfs_obj.user_path + 'FlightRadarApi/zones'
# destination_path = hdfs_obj.base_url + 'FlightRadarApi/zones'
print(destination_path)

In [ ]:
zones_df.count()

In [ ]:
zones_df.write.parquet(destination_path, mode='overwrite')

In [ ]:
spark.stop()